## Reading the libraries

In [1]:
import numpy as np
import pandas as pd
import os
import os.path
from operator import add
from datetime import datetime, date, timedelta
from matplotlib import pyplot as plt


In [2]:
pd.set_option('display.max_columns',69)
pd.set_option('display.max_rows',138)

## Step 1: Climate Scenarios (CH2018, and Random Scenarios)

In [25]:
def is_leap(year):
    """ return true for leap years, False for non leap years """
    return year % 4 == 0 and ( year % 100 != 0 or year % 400 == 0)


def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [38]:
def random_pcp(dfpcp,FirstYear, LastYear, ScenarioNumbers, RCPNames):
    
    #outFileName = OutFileName
    dfpcpCol = dfpcp.columns
    #sceNum = len(dfCol)

    sceNum = ScenarioNumbers
    firstYear = FirstYear
    lastYear = LastYear
    simLen = lastYear - firstYear + 1

    from random import shuffle
    a = []
    for x in range(simLen): 
        randomInd = [z for z in range(sceNum)]
        shuffle(randomInd)
        a.append(randomInd)
        
    RCP = RCPNames
    columnsDfpcp = ['sc_' + RCP + str(k) for k in range(1, sceNum+1,1)] 
    c = [[19810101 for p in range(sceNum)]]
    #df1 = 'df' + str(outDFNumber)
    df1pcp = pd.DataFrame(c, columns =columnsDfpcp)
    #df1.to_csv('SAeidVaghefimodified1111222.csv', index = False)

    c.clear()

    i = 0
    p = 1
    for m in range(firstYear, lastYear + 1, 1):
        if is_leap(m):
            for j in range(((m - firstYear)*365+p) , ((m - firstYear)*365+367+p-1), 1):
                c.append(dfpcp[dfpcpCol[a[i]]].iloc[j].values) 
            i += 1 # counter i; equal to simulation length (simLen)
            p += 1

        else:
            for j in range(((m - firstYear)*365+p), ((m - firstYear)*365+366+p-1), 1):
                c.append(dfpcp[dfpcpCol[a[i]]].iloc[j].values) 
            i += 1
                
        print(m)

        dfnewpcp = 'df' + str(m)
        dfnewpcp = pd.DataFrame(c, columns =columnsDfpcp)
        c.clear()
        df1pcp = df1pcp.append(dfnewpcp, ignore_index=True)
        #df1.to_csv('SAmodified1111222.csv', mode='w', header=True, index = False)
    return df1pcp

In [39]:
def random_tmp(dftmp,FirstYear, LastYear, ScenarioNumbers, RCPNames):

    #dfCol = df.columns
    #sceNum = len(dfCol) // 2
    sceNum = ScenarioNumbers
    firstYear = FirstYear
    lastYear = LastYear
    simLen = lastYear - firstYear + 1

    dftmpColMax = dftmp.columns[[i for i in range(0, sceNum*2, 2)]]
    dftmpColMin = dftmp.columns[[i for i in range(1, sceNum*2, 2)]]

    ## yek list be toole 119 ke dakhelesh list haye 68 ta ee darim be soorate random
    from random import shuffle
    a = []
    for i in range(simLen): 
        randomInd = [j for j in range(sceNum)]
        #x = [[i] for i in range(10)] 
        shuffle(randomInd)
        a.append(randomInd)
        
    #print('end!')

    cT = []
    RCP = RCPNames
    columnsDfOdd = ['sc_' + RCP + str(k)  for k in range(1, sceNum+1,1)] 
    columnsDfEven = [""] * sceNum

    columnsDftmp = []
    #colOdd = ['Scr_' + str(i) for i in range(1, sceNum+1, 1)]
    #colEven = ['' for i in range(1, sceNum+1,1)]

    for i in range (sceNum):
        columnsDftmp.append(columnsDfOdd[i])
        columnsDftmp.append(columnsDfEven[i])


    #### OR:
    #columnsDf = ["Sr", ""] * sceNum
    df1tmp = pd.DataFrame(cT, columns =columnsDftmp)
    #df1.to_csv("rrrrrrrrmodified1111222.csv", index = False)


    cMax = [[19810101 for p in range(sceNum)]]
    cMin = [["" for p in range(sceNum)]]
    i = 0
    p = 1
    for m in range(firstYear, lastYear + 1, 1):
        if is_leap(m):
            for j in range(((m - firstYear)*365+p) , ((m - firstYear)*365+367+p-1), 1):
                cMax.append(dftmp[dftmpColMax[a[i]]].iloc[j].values)
                cMin.append(dftmp[dftmpColMin[a[i]]].iloc[j].values) 
    
            i += 1

        else:
            for j in range(((m - firstYear)*365+p), ((m - firstYear)*365+366+p-1), 1):
                cMax.append(dftmp[dftmpColMax[a[i]]].iloc[j].values)
                cMin.append(dftmp[dftmpColMin[a[i]]].iloc[j].values)

            i += 1

        c = []
        for y in range(0, len(cMax), 1): # the length of simulation years
            for z in range(sceNum): # range(4)
                c.append(cMax[y][z])
                c.append(cMin[y][z])

        cMax.clear()
        cMin.clear()

        cMain = []
        cMain = list(chunks(c, sceNum * 2))
        print(m)

    ### Should be checked

        dfnewtmp = 'dftmp' + str(m)
        #columnsDf = ["Sr", ""]*sceNum
        #columnsDf = [['sc_' + str(k), ""] for k in range(1, sceNum+1,1)] 
        dfnewtmp = pd.DataFrame(cMain, columns =columnsDftmp)
        c.clear()
        df1tmp = df1tmp.append(dfnewtmp, ignore_index=True)
        #df1.to_csv("rrrrrrrrmodified1111222.csv", mode='w', header=True, index = False)
    return df1tmp

In [40]:
def randomness_pcp_tmp(fnames):
    for f in fnames:
        if 'p.csv' in f:
            print('pcp files start working!')
            #df = pd.read_csv('47-0625000_8-6666667p.csv')
            dfpcp = pd.read_csv(f)


            filt1 = dfpcp.columns.str.contains('RCP26') #12
            filt2 = dfpcp.columns.str.contains('RCP45') #25
            filt3 = dfpcp.columns.str.contains('RCP85') #31

            dfpcpRCP26 = dfpcp.loc[:, filt1]
            dfpcpRCP45 = dfpcp.loc[:, filt2]
            dfpcpRCP85 = dfpcp.loc[:, filt3]

            dfpcpRCP26_n = random_pcp(dfpcpRCP26, 1981, 2099, 12, '26_')
            dfpcpRCP45_n = random_pcp(dfpcpRCP45, 1981, 2099, 25, '45_')
            dfpcpRCP85_n = random_pcp(dfpcpRCP85, 1981, 2099, 31, '85_')


            result = pd.concat([dfpcpRCP26_n, dfpcpRCP45_n, dfpcpRCP85_n], axis=1, sort=False)
            #result.to_csv('47-0625000_8-6666667p_n1.csv', index = False)


            #newName = 'n_'+ f
            newName = f
            #filepath = os.path.join(os.getcwd(), newName)
            root = os.getcwd()
            if os.path.isdir(os.path.join(root, 'Outputs_randomness')):
                pass
            else: os.mkdir(os.path.join(root, 'Outputs_randomness'))

            outfolder = os.path.join(os.getcwd(), 'Outputs_randomness')
            filepath = os.path.join(outfolder, newName)

            result.to_csv(filepath, index = False)
            print('end!')
            print("--- %s seconds ---" % (time.time() - start_time))


        elif 't.csv' in f:
            print('tmp files start working!')
            dftmp = pd.read_csv(f)
            dftmpCol = list(dftmp.columns)

            filt1_max = [dftmpCol.index(s) for s in dftmpCol if "RCP26" in s]
            filt2_max = [dftmpCol.index(s) for s in dftmpCol if "RCP45" in s]
            filt3_max = [dftmpCol.index(s) for s in dftmpCol if "RCP85" in s]

            aOnefilt1= [1]*len(filt1_max)
            aOnefilt2= [1]*len(filt2_max)
            aOnefilt3= [1]*len(filt3_max)

            filt1_min = list(map(add, filt1_max, aOnefilt1))
            filt2_min = list(map(add, filt2_max, aOnefilt2))
            filt3_min = list(map(add, filt3_max, aOnefilt3))

            filt1Tot = []
            for i in range(len(filt1_max)):
                filt1Tot.append(filt1_max[i])
                filt1Tot.append(filt1_min[i])

            filt2Tot = []
            for j in range(len(filt2_max)):
                filt2Tot.append(filt2_max[j])
                filt2Tot.append(filt2_min[j])

            filt3Tot = []
            for k in range(len(filt3_max)):
                filt3Tot.append(filt3_max[k])
                filt3Tot.append(filt3_min[k])

            dftmpRCP26 = dftmp[dftmp.columns[filt1Tot]]
            dftmpRCP45 = dftmp[dftmp.columns[filt2Tot]]
            dftmpRCP85 = dftmp[dftmp.columns[filt3Tot]]

            dftmpRCP26_n = random_tmp (dftmpRCP26, 1981, 2099, 12, '26_')
            dftmpRCP45_n = random_tmp (dftmpRCP45, 1981, 2099, 25, '45_')
            dftmpRCP85_n = random_tmp (dftmpRCP85, 1981, 2099, 31, '85_')

            result = pd.concat([dftmpRCP26_n, dftmpRCP45_n, dftmpRCP85_n], axis=1, sort=False)

            #ewName = 'n'+f
            #ilepath = os.path.join(os.environ.get('HOME'), newName)
            #esult.to_csv(filepath, index = False)

            #newName = 'n_'+ f
            newName = f
            #filepath = os.path.join(os.getcwd(), newName)

            outfolder =os.path.join(os.getcwd(), 'Outputs_randomness')
            filepath = os.path.join(outfolder, newName)
            result.to_csv(filepath, index = False)

            print('end!')

        else :
            pass

pcp files start working!
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056


NameError: name 'time' is not defined

In [29]:
rootFolder = r'C:\Users\ashrafse\SA_2\snowModelUZH\case3_hoch-ybrig'
inputFolder = os.path.join(rootFolder,'input')
climate_ref_Folder = os.path.join(inputFolder, 'Climate_ref')

In [30]:
os.chdir(climate_ref_Folder)

In [32]:
fnames = os.listdir()

In [33]:
fnames

['47-0000000_8-7708333p.csv',
 '47-0000000_8-7708333t.csv',
 'pcp.txt',
 'tmp.txt']

In [ ]:
randomness_pcp_tmp(fnames)

## Step2: function for initiating the main dictionary of climate stations

In [3]:
def create_dic(a):
    '''Function: creating a dictionary for each climate station'''
    
    a = {}
    keys = ['fM', 'iPot', 'rSnow', 'dSnow', 'cPrec', 'dP', 'elev', 'lat', 'long', 'fileName']
    a = {key: None for key in keys}
    return a

def initialize_input_dict (mainFolderSki):
    ''' This function returns a dictionary , and addresses of 4 folders'''
    
    
    '''Step 1''' 
    rootFolder = mainFolderSki
    inputFolder = os.path.join(rootFolder,'input')
    ablationFolder = os.path.join(inputFolder, 'Ablation')
    accumulationFolder = os.path.join(inputFolder, 'Accumulation')
    climate_ref_Folder = os.path.join(inputFolder, 'Climate_ref')
    
    
    '''Step 2: Reading all files names inside the Ablation, Accumulation, and Climate folders'''  
    ablationFiles = []
    for filename in os.walk(ablationFolder):
        ablationFiles = filename[2]
    
    accumulationFiles = list()
    for filename in os.walk(accumulationFolder):
        accumulationFiles = filename[2]

    climate_ref_Files = list()
    for filename in os.walk(climate_ref_Folder):
        climate_ref_Files = filename[2]
        
        
    '''Step 3: Reading files inside ablation folder '''
    os.chdir(ablationFolder)
    with open(ablationFiles[0], 'r') as file:
        FM1 = file.read()
    with open(ablationFiles[1], 'r') as file:
        Ipot1 = file.read()
    with open(ablationFiles[2], 'r') as file:
        Rsnow1 = file.read()
        
        
    '''Step 4: Reading the lines of files inside ablation folder'''
    FM1 = FM1.replace('\n', '\t')
    FM1 = FM1.split('\t')
    Ipot1 = Ipot1.replace('\n', '\t').split('\t')
    Rsnow1 = Rsnow1.replace('\n', '\t').split('\t')
        
        
    '''Step 5: Reading the lines of files inside accumulation folder''' 
    os.chdir(accumulationFolder)
    
    with open(accumulationFiles[0], 'r') as file:
        cPrec = file.read()
    with open(accumulationFiles[1], 'r') as file:
        dSnow1 = file.read()
    
    cPrec = cPrec.replace('\n', '\t')
    cPrec = cPrec.split('\t')
    dSnow1 = dSnow1.replace('\n', '\t').split('\t')
    
    
    '''Step 6: Reading the lines of files inside climate folder''' 
    os.chdir(climate_ref_Folder)
    
    with open('pcp.txt', 'r') as file:
        pcpData = file.read()
    with open('tmp.txt', 'r') as file:
        tmpData = file.read()
        
    pcpData = pcpData.split('\n')
    
    for i in range(len(pcpData)):
        pcpData[i] = pcpData[i].split(',')
        
        
    '''Step 7: Initialazing the input dictionary of climate stations which holds the information of accumulation
     and ablation, and etc of the stations''' 
    nameStn = []
    for file in climate_ref_Files:
        if 'p.csv' in file:
            #nameStn.append('n_' + file[-25: -5])
            nameStn.append(file[-25: -5])

    stnDicts = []
    for i in range(len(nameStn)):
        stnDicts.append(create_dic(nameStn[i]))
    
    
    '''Step 8: Assigning the file names to the dictionary'''
    for i in range (len(nameStn)):
        stnDicts[i]['fileName'] = nameStn[i]

    
    '''Step 9: Assigning the accumulation and ablation values'''
    for stnDict in stnDicts:
        for i, element in enumerate(FM1):
            if element == stnDict['fileName'][:]:
            #if element == stnDict['fileName'][2:]:
                stnDict['fM'] = FM1[i+1]
                
        for i, element in enumerate(Ipot1):
            if element == stnDict['fileName'][:]:
            #if element == stnDict['fileName'][2:]:
                stnDict['iPot'] = Ipot1[i+1]

        for i, element in enumerate(Rsnow1):
            if element == stnDict['fileName'][:]:
            #if element == stnDict['fileName'][2:]:  
                stnDict['rSnow'] = Rsnow1[i+1]

        for i, element in enumerate(dSnow1):
            if element == stnDict['fileName'][:]:
            #if element == stnDict['fileName'][2:]:
                stnDict['dSnow'] = dSnow1[i+1]

        for i, element in enumerate(cPrec):
            stnDict['cPrec'] = cPrec[1]
            stnDict['dP'] = cPrec[3]
            
    '''Step 10: Assigning the elevation, Lat and long to the dictionaries'''
    for i in range(len(stnDicts)):
        for j in range(1, len(pcpData)):
            
            #if pcpData[j][1][2:-1] == stnDicts[i]['fileName'][2:]:
            if pcpData[j][1][:-1] == stnDicts[i]['fileName'][:]:
                stnDicts[i]['lat']= pcpData[j][2]
                stnDicts[i]['long']= pcpData[j][3]
                stnDicts[i]['elev']= pcpData[j][4]
                
    return stnDicts, inputFolder, ablationFolder, accumulationFolder, climate_ref_Folder

## Step3: Initializiing the main dictionary for a case study

In [4]:
caseStudyStns = {}
inputFolder = ''
ablationFolder = ''
accumulationFolder = ''
climateFolder = ''
#root = 'C:/Users/ashrafse/SA_2/snowModelUZH/case2_Atzmaening'
#root = 'C:/Users/ashrafse/SA_2/snowModelUZH/case6_davos_elevations'
root = r'C:\Users\ashrafse\SA_2\snowModelUZH\case3_hoch-ybrig'

## calling the function with multiple return values
caseStudyStns, inputFolder, ablationFolder, accumulationFolder, climateFolder = initialize_input_dict(root)

In [5]:
caseStudyStns

[{'fM': '1.012',
  'iPot': '1000',
  'rSnow': '0.5',
  'dSnow': '0.5',
  'cPrec': '0',
  'dP': '0',
  'elev': '1755',
  'lat': '47.00',
  'long': '8.7708333',
  'fileName': '47-0000000_8-7708333'}]

## Step4: Matching the station names values in the dictionary of stations with CSV files in Climate folder of the case Study

In [6]:
#pcpAtzmaening = []
#tmpAtzmaening = []
pcpCaseStudy = []
tmpCaseStudy = []

for i in range(len(caseStudyStns)):
    pcpCaseStudy.append(os.path.join(climateFolder, caseStudyStns[i]['fileName'] + 'p.csv'))
    tmpCaseStudy.append(os.path.join(climateFolder, caseStudyStns[i]['fileName'] + 't.csv'))

In [7]:
tmpCaseStudy

['C:\\Users\\ashrafse\\SA_2\\snowModelUZH\\case3_hoch-ybrig\\input\\Climate_ref\\47-0000000_8-7708333t.csv']

In [8]:
pcpCaseStudy

['C:\\Users\\ashrafse\\SA_2\\snowModelUZH\\case3_hoch-ybrig\\input\\Climate_ref\\47-0000000_8-7708333p.csv']

## Step5: building a database for each precipitation and temperature file in Climate folder and saving them in a list

In [9]:
dfpcp = [None for _ in range(len(pcpCaseStudy))]
dftmp = [None for _ in range(len(tmpCaseStudy))]
for i in range(len(pcpCaseStudy)):
    dfpcp[i] = pd.read_csv(pcpCaseStudy[i])
    dftmp[i] = pd.read_csv(tmpCaseStudy[i])

#### making a header for output files

In [10]:
dfpcpCol = dfpcp[0].columns

In [11]:
dftmpCol = dftmp[0].columns

#### defining the length of simulations and scenarios

In [12]:
scenariosLength = len(dfpcpCol)

In [13]:
simulationLength = len(dftmp[0][dftmpCol[0]]) - 1

## Step 5: Main Snow Model

In [14]:
## 1st column as index: makaing date from 01 01 1981 to 2099 12 31
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date ).days + 1)):
        yield start_date + timedelta(n)

In [18]:
#def snow_Model (caseStudyStns, dateList, seasonList, thershold = None):
#def snow_Model (caseStudyStns, dateList, seasonList, x1 = None):
def snow_Model (x1 = None):

    
    caseStudyStns = {}
    inputFolder = ''
    ablationFolder = ''
    accumulationFolder = ''
    climateFolder = ''
    #root = 'C:/Users/ashrafse/SA_2/snowModelUZH/case2_Atzmaening'
    #root = 'C:/Users/ashrafse/SA_2/snowModelUZH/case6_davos_elevations'
    root = r'C:\Users\ashrafse\SA_2\snowModelUZH\case3_hoch-ybrig'

    ## calling the function with multiple return values
    caseStudyStns, inputFolder, ablationFolder, accumulationFolder, climateFolder = initialize_input_dict(root)
    
    
    
    ## Reading the beginning and end of the simulation
    start_date = date(1981, 1, 1)
    end_date = date(2099, 12, 31)
    dateList = []
    for single_date in daterange(start_date, end_date):
        dateList.append(single_date.strftime("%m/%d/%Y"))

    seasonList = []
    for n in range (1981, 2100, 1):
        seasonList.append(str(n))
    
    
    
    ##### PART 1 : daily outputs
    ## Running the model for each climate station:
    
    for k in range(len(caseStudyStns)):

        ## making a header for output files
        dfpcpCol = dfpcp[k].columns
        dftmpCol = dftmp[k].columns


        ## defining the length of simulations and scenarios
        scenariosLength = len(dfpcpCol)
        simulationLength = len(dftmp[0][dftmpCol[0]]) - 1


        ## declaring the initial arrays
        accumulation = [0 for _ in range(simulationLength)]
        ablation =  [0 for _ in range(simulationLength)]
        snowDeposite = [0 for _ in range(simulationLength)]
        total = np.zeros([simulationLength, 3*scenariosLength])


        ## Running the model for each climate scenario:
        for j in range(len(dfpcpCol)):
            ## Reading the information and inputs of the first day of simulation
            todayPCP = dfpcp[k][dfpcpCol[j]].iloc[1] if (dfpcp[k][dfpcpCol[j]].iloc[1] != -99) else 0
            todayTMPMAX = round(dftmp[k][dftmpCol[2*j]].iloc[1],2) if(dftmp[k][dftmpCol[2*j]].iloc[1] != -99) else 0
            todayTMPMIN = round(dftmp[k][dftmpCol[2*j+1]].iloc[1],2) if(dftmp[k][dftmpCol[2*j+1]].iloc[1] != -99) else 0
            todayTMPAVE = round((todayTMPMAX+todayTMPMIN)/2,2) if((todayTMPMAX+todayTMPMIN)/2 != -99) else 0


            ### Accumulation for the first day:
            if (todayTMPAVE) <=-1:
                accumulation[0] = todayPCP *(1 + float(caseStudyStns[k]['cPrec']))*float(caseStudyStns[k]['dSnow'])*(1)

            elif -1 < (todayTMPAVE) <= 1:
                accumulation[0] = todayPCP *(1 + float(caseStudyStns[k]['cPrec']))*float(caseStudyStns[k]['dSnow'])*float((1-todayTMPAVE)/2)

            else: accumulation[0] = 0


            ### Ablation for the first day:
            if todayTMPAVE <= 1:
                 ablation[0] = 0
            else: 
                ablation[0] = (float(caseStudyStns[k]['fM']) + float(caseStudyStns[k]['rSnow'])*float(caseStudyStns[k]['iPot'])*0.001)*float(todayTMPAVE)*(1+0)

            ### Main mass balance equation for the first day:
            snowDeposite[0] = 0 if (0 + accumulation[0] - ablation[0]) < 0 else (0 + accumulation[0] - ablation[0])

            ### storing three values in a list for the first day
            total[0,3*j+0] = round((accumulation[0] - ablation[0]), 2)
            total[0,3*j+1] = round(snowDeposite[0], 2)
            total[0,3*j+2] = 1 if (total[0,3*j+1] > x1) else 0


            ## For the second day to the end of simulation:
            i = 0
            for i in range(2, simulationLength + 1, 1):
                # precipitation and temperature missing values were handled

                ## Reading the information and inputs of the first day of simulation
                todayPCP = dfpcp[k][dfpcpCol[j]].iloc[i] if (dfpcp[k][dfpcpCol[j]].iloc[i] != -99) else 0
                todayTMPMAX = round(dftmp[k][dftmpCol[2*j]].iloc[i],2) if(dftmp[k][dftmpCol[2*j]].iloc[i] != -99) else 0
                todayTMPMIN = round(dftmp[k][dftmpCol[2*j+1]].iloc[i],2) if(dftmp[k][dftmpCol[2*j+1]].iloc[i] != -99) else 0
                todayTMPAVE = round((todayTMPMAX+todayTMPMIN)/2,2) if((todayTMPMAX+todayTMPMIN)/2 != -99) else 0

                ### Accumulation :
                if(todayTMPAVE) <= -1:
                    ##
                    accumulation[i-1] = todayPCP *(1 + float(caseStudyStns[k]['cPrec']))*float(caseStudyStns[k]['dSnow'])*(1)

                elif -1 < (todayTMPAVE) <= 1:
                    accumulation[i-1] = todayPCP *(1 + float(caseStudyStns[k]['cPrec']))*float(caseStudyStns[k]['dSnow'])*float((1-todayTMPAVE)/2)

                else: accumulation[i-1] = 0

                ### Ablation :
                if todayTMPAVE <= 0:
                    ablation[i-1] = 0
                else: 
                    ablation[i-1] = (float(caseStudyStns[k]['fM']) + float(caseStudyStns[k]['rSnow'])*float(caseStudyStns[k]['iPot'])*0.001)*float(todayTMPAVE)*(1+0)

                ### Main mass balance equation for second day to the end of simulation:
                snowDeposite[i-1] = 0 if (snowDeposite[i-2] + accumulation[i-1] - ablation[i-1]) < 0 else (snowDeposite[i-2] + accumulation[i-1] - ablation[i-1])


                ### storing three values in a list 
                total[i-1,3*j+0] = round((accumulation[i-1] - ablation[i-1]) , 2)
                total[i-1,3*j+1] = round(snowDeposite[i-1], 2)
                total[i-1,3*j+2] = 1 if (total[i-1,3*j+1] > x1) else 0


        ### Saving the output of total list in a csv file in a specific path

        ## 1st row as the column names:
        columnsDF = [] 
        for col in dfpcpCol:
            columnsDF.append('SnowAmount_' + col)
            columnsDF.append('TotalSnowAmount_' + col)
            columnsDF.append('isOverSnow_' + col)


        columnsDF0 = ['DATE']
        dfnew0 = pd.DataFrame(dateList, columns = columnsDF0)
        dfnew = pd.DataFrame(total, columns = columnsDF)
        df1 = pd.concat([dfnew0, dfnew], axis=1, sort=False)

        if os.path.isdir(os.path.join(root, 'Outputs_py')):
            pass
        else: os.mkdir(os.path.join(root, 'Outputs_py'))

        outfolder =os.path.join(root, 'Outputs_py') 
        outfileName = 'Total_daily_' + caseStudyStns[k]['fileName'] + '.csv'
        outputFile = os.path.join(outfolder, outfileName )
        df1.to_csv(outputFile, index = False)
        #return df1
        
        
        
        
        ##### PART 2 seasonal outputs
        
        total_Daily_Files = list()
        for filename in os.walk(outfolder):
            total_Daily_Files = filename[2]
            
        ##get just total nor existing seasonal...
        
        totalFiles = []
        for i in range(len(total_Daily_Files)):
            totalFiles.append(os.path.join(outfolder, total_Daily_Files[i]))
        
        totalFiles = []
        for i in range(len(total_Daily_Files)):
            if 'season' in total_Daily_Files[i]:
                continue
            else: totalFiles.append(os.path.join(outfolder, total_Daily_Files[i]))
                
        
        dfSeason = [ None for _ in range(len(totalFiles))]
        for i in range(len(totalFiles)):
            dfSeason[i] = pd.read_csv(totalFiles[i], low_memory=False)

            start_date = date(1981, 1, 2)
            end_date = date(2099, 12, 31)
            dateList = []
            for single_date in daterange(start_date, end_date):
                dateList.append(single_date.strftime("%m/%d/%Y"))

            start_season = []
            end_season = []

            for pp in range (1981, 2099, 1):
                #start_season.append(date(i, 1, 1))
                #end_season.append(date(i+1, 12, 31)) if i != 2099 else end_season.append(date(i, 12, 30))
                start_season.append(date(pp, 11, 1))
                end_season.append(date(pp+1, 4, 30))

            df2 = dfSeason[i]
            df2.set_index('DATE', inplace = True)
            df2Col = df2.columns

            df2ColCal = []
            for m in range(68):
            #for k in range(68):
                df2ColCal.append(df2Col[3*m+2])

            sumGoodCondition = np.zeros([len(start_season), len(df2ColCal)])


            for j in range(len(df2ColCal)):
                for k in range(len(start_season)):
                #for i in range(3):
                    start_date = start_season[k]
                    end_date = end_season[k]
                        #start_date = date(1981, 1, 2)
                        #end_date = date(1981, 1, 10)
                    for single_date in daterange(start_date, end_date):
                        #sumGood1[i] += df2['TotalSnowAmount_CLMCOM-CCLM4-ECEARTH-EUR11-RCP45-pcp'].loc[single_date.strftime("%m/%d/%Y")]
                        sumGoodCondition[k,j] += df2[df2ColCal[j]].loc[single_date.strftime("%m/%d/%Y")]
                            #print(single_date.strftime("%Y-%m-%d"))

            df3 = pd.DataFrame(sumGoodCondition, columns = df2ColCal)


            firstCol = []
            for o in range (len(seasonList)-1):
                firstCol.append(seasonList[o] +'-' + seasonList[o+1])

            columnsDF1 = ['Season']
            dfnew0 = pd.DataFrame(firstCol, columns = columnsDF1)

            dfFinalSeason = pd.concat([dfnew0, df3], axis=1, sort=False)

            outfileNameSeason = 'season_' + total_Daily_Files[i]
            outputFile = os.path.join(outfolder, outfileNameSeason)
            dfFinalSeason.to_csv(outputFile, index = False)

            
        
        #return df1, outfolder, dfFinalSeason
        return {'y': x1*2}

## Step 6: EMA_Workbench connector

In [19]:
'''
Created on 20 dec. 2010

This file illustrated the use the EMA classes for a contrived example
It's main purpose has been to test the parallel processing functionality

.. codeauthor:: jhkwakkel <j.h.kwakkel (at) tudelft (dot) nl>
'''
from __future__ import (absolute_import, print_function, division,
                        unicode_literals)

from ema_workbench import (Model, RealParameter, ScalarOutcome, ema_logging,
                           perform_experiments)

In [20]:
if __name__ == '__main__':
    ema_logging.LOG_FORMAT = '[%(name)s/%(levelname)s/%(processName)s] %(message)s'
    ema_logging.log_to_stderr(ema_logging.INFO)

    #model = Model('simpleModel', function=some_model)  # instantiate the model
    model = Model('simpleModel', function = snow_Model)  # instantiate the model

    # specify uncertainties
    model.uncertainties = [RealParameter("x1", 100.0, 300.0)]
    
    #model.uncertainties = [RealParameter("x1", 0.1, 10),
     #                      RealParameter("x2", -0.01, 0.01),
      #                     RealParameter("x3", -0.01, 0.01)]
   

    # specify outcomes
    #model.outcomes = [ScalarOutcome('y')]

    results = perform_experiments(model, 2)

[EMA.ema_workbench.em_framework.evaluators/INFO/MainProcess] performing 2 scenarios * 1 policies * 1 model(s) = 2 experiments
[EMA.ema_workbench.em_framework.evaluators/INFO/MainProcess] performing experiments sequentially
[EMA.ema_workbench.em_framework.callbacks/INFO/MainProcess] 1 cases completed
[EMA.ema_workbench.em_framework.callbacks/INFO/MainProcess] 2 cases completed
[EMA.ema_workbench.em_framework.evaluators/INFO/MainProcess] experiments finished


In [21]:
results

(           x1 scenario policy        model
 0  143.215471        2   None  simpleModel
 1  246.907592        3   None  simpleModel, {})